# Computing the volume underneath a Pareto frontier
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes the Pearson correlation coefficients between each pair of objectives. This provides a measure of the conflict among the objectives.

In [35]:
import pandas as pd
import numpy as np
import itertools

### Prepocess data and normalize the objective space

In [155]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/tothMcDill_2009/tothMcDill_2009.txt")

In [156]:
# Hard-coded manipulations to tidy-up sols

#sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns (climate change sets)
#sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column (pack forest and Chilean sets and portfolio opt)
# nothing for some sets (toth mcDill, small sed fire)

In [157]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [158]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [159]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 1
#objSenses["Northern Spotted Owl Habitat (ha)"] = 1 #for climate change sets (all others 0)
objSenses["PER"] = 0 # for tothMcDill set (all others 1)
#objSenses['MaxSediment'] = 0
#objSenses[" Risk"] = 0
#All 1 for PackForest sets

In [160]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [161]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [162]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [163]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [164]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Computing the Pearson correlation coefficients

In [165]:
# Compute correlation coefficients for each combination of objectives


for objPair in itertools.combinations(objs,2):
    print(objPair[0] + ", " + objPair[1] + ", " + \
          str(np.corrcoef(sols[objPair[0]],sols[objPair[1]])[0,1]))

NPV, HAB, -0.833379085831
NPV, PER, 0.567336358456
HAB, PER, -0.90286787026
